# Work in Progress: Load Parquet Into Redshift

## Check: 

https://docs.aws.amazon.com/redshift/latest/dg/r_CREATE_EXTERNAL_TABLE.html for loading Parquet into Redshift via Spectrum


In [ ]:
!pip install -q SQLAlchemy==1.3.13

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import pandas as pd

In [1]:
# Redshift configuration
redshift_username = 'dsoaws'
redshift_pw = ''

# TODO: get Endpoint/Host name programatically
redshift_host = ''
redshift_database = 'dsoaws'
redshift_port = '5439'

schema = 'redshift'
table_name_tsv = 'amazon_reviews_tsv'
table_name_parquet = 'amazon_reviews_parquet'

In [ ]:
# Connect to Redshift database engine
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(redshift_username, redshift_pw, redshift_host, redshift_port, redshift_database))


In [ ]:
# Configure Session
session = sessionmaker()
session.configure(bind=engine)
s = session()
#set_path = "SET search_path TO %s" % schema
s.execute(set_path)

## Load Parquet Data From S3 Into Redshift

### Create Parquet Table In Redshift

In [ ]:
statement = """DROP TABLE IF EXISTS {}""".format(table_name_parquet)

s = session()
s.execute(statement)

In [ ]:
#          product_category varchar(24)
statement = """CREATE TABLE {}( 
         marketplace varchar(2),
         customer_id varchar(8),
         review_id varchar(14),
         product_id varchar(10),
         product_parent varchar(9),
         product_title varchar(400),
         star_rating int,
         helpful_votes int,
         total_votes int,
         vine varchar(1),
         verified_purchase varchar(1),
         review_headline varchar(128),
         review_body varchar(65535),
         review_date varchar(10),
         year int
)""".format(table_name_parquet)

print(statement)

In [ ]:
s.execute(statement)
s.commit()

In [ ]:
# "/" trainling slash in FROM Path is required not to pick up another directory in S3 path
# NOTE: this does not work -- use Spectrum to load parquet data

statement = """COPY {}
                FROM '{}/'
                IAM_ROLE '{}' 
                FORMAT AS PARQUET;""".format(table_name_parquet, s3_destination_path_parquet, IAM_ROLE)
print(statement)

In [ ]:
s = session()
s.execute(statement)
s.commit()

### Run another sample query

In [ ]:
df = pd.read_sql_query("""SELECT product_id, product_title, count(*) AS count_reviews
                            FROM \'{}\'
                            WHERE product_category=\'Digital_Video_Download\'
                            GROUP BY product_id, product_title
                            ORDER BY count_reviews DESC""".format(table_name_parquet), engine)

In [ ]:
df.head(5)